pip install pandas scikit-learn xgboost lightgbm matplotlib seaborn


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
# ========== 1. Load Dataset ==========
df = pd.read_csv("2015.csv")  # Use a single year for simplicity

In [ ]:
# ========== 2. Select Useful Columns ==========
df = df[[
    'MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'CRS_DEP_TIME',
    'DISTANCE', 'DEP_DELAY', 'ARR_DELAY'
]]

In [ ]:
# ========== 3. Target Creation ==========
df = df.dropna(subset=['DEP_DELAY', 'ARR_DELAY'])  # Drop rows with missing delay
df['Delayed'] = (df['ARR_DELAY'] > 15).astype(int)  # Delayed if arrival delay > 15 mins